In [124]:
import os
os.environ["XDG_CACHE_HOME"] = "/home/olab/tomerronen1/xdg_cache"
os.environ["AUTH_TOKEN"] = "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"
import datasets
dataset_dict = datasets.load_dataset("hotpot_qa", "distractor")
import torch

Using the latest cached version of the module from /home/olab/tomerronen1/xdg_cache/huggingface/modules/datasets_modules/datasets/hotpot_qa/133b9501f892e5193babbad937bee3b4899deb4691ef4d791e6ac0111c875bb5 (last modified on Thu Sep 22 14:56:17 2022) since it couldn't be found locally at /home/olab/tomerronen1/git_repos/last_projects_playground/long_rangeness/hotpot_qa/hotpot_qa.py, or remotely (FileNotFoundError).
Reusing dataset hotpot_qa (/home/olab/tomerronen1/xdg_cache/huggingface/datasets/hotpot_qa/distractor/1.0.0/133b9501f892e5193babbad937bee3b4899deb4691ef4d791e6ac0111c875bb5)


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
example = dataset_dict["train"][6]
print(example["question"])
print(example["answer"])
print(example["supporting_facts"])
for title, sentence_id in zip(example["supporting_facts"]["title"], example["supporting_facts"]["sent_id"]):
    paragraph_id = example["context"]["title"].index(title)
    print(example["context"]["sentences"][paragraph_id][sentence_id])

Which genus of moth in the world's seventh-largest country contains only one species?
Crambidae
{'title': ['Indogrammodes', 'Indogrammodes', 'India', 'India'], 'sent_id': [0, 1, 0, 1]}
Indogrammodes is a genus of moths of the Crambidae family.
 It contains only one species, Indogrammodes pectinicornalis, which is found in India.
India, officially the Republic of India ("Bhārat Gaṇarājya"), is a country in South Asia.
 It is the seventh-largest country by area, the second-most populous country (with over 1.2 billion people), and the most populous democracy in the world.


In [3]:
import sled
from transformers import AutoModelForSeq2SeqLM, AutoConfig
model_name = "tau/bart-base-sled-govreport"
auth_token = "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"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, use_auth_token=auth_token)


Downloading:   0%|          | 0.00/206 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/558M [00:00<?, ?B/s]

There were unexpected keys in the checkpoint model loaded: ['_lm_head.weight'].


In [14]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=auth_token)

Downloading:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
import torch
cache = []

def append_grad(grad):
    cache.append(grad.detach().cpu().numpy())

emb_tensor = model._underlying_model.model.shared.weight
emb_tensor.register_hook(append_grad)

In [17]:
from datasets import load_dataset
dataset = load_dataset("ccdv/govreport-summarization", split="validation[:10]")

No config specified, defaulting to: gov_report_summarization_dataset/document
Reusing dataset gov_report_summarization_dataset (/home/olab/tomerronen1/xdg_cache/huggingface/datasets/ccdv___gov_report_summarization_dataset)/document/1.0.0/57ca3042de9c40c218cc94084cbc80a99a161036134bfc88112c57d251443590)


In [113]:
batch = tokenizer(dataset[:2]["report"], return_tensors='pt', padding=True, truncation=True, max_length=200)
gen_batch = model.generate(**batch, num_beams=1, max_length=30)
decoder_attention_mask = (gen_batch != tokenizer.pad_token_id).long()
model_output = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"],
                     decoder_input_ids=gen_batch, decoder_attention_mask=decoder_attention_mask)

/home/olab/tomerronen1/miniconda3/envs/mlskel/lib/python3.9/site-packages/sled/modeling_sled.py:693: UserWarning: prefix_length is missing from kwargs_tensor_keys (though expected for SLED prefix prepending)
  warnings.warn(f'{k} is missing from kwargs_tensor_keys (though expected for SLED prefix prepending)')


In [114]:
logits = model_output.logits[:,:-2,:]  # without additional token [-1], without the token that replaces the forced EOS [-2].
generated_tokens = gen_batch[:,1:-1]  # without the EOS that starts the generation, without the forced EOS at the end.
manual_greedy_output = logits.argmax(-1)
assert (manual_greedy_output == generated_tokens).all()

In [133]:
# logprobs = logits.log_softmax(dim=-1)  # mote correct, but then everything has gradient - is that good?
logprobs = logits
generated_logprobs = torch.gather(input=logprobs, dim=-1, index=generated_tokens.unsqueeze(-1)).squeeze(-1)
manual_greedy_generated_logprobs = logprobs.max(dim=-1).values
assert (generated_logprobs == manual_greedy_generated_logprobs).all()

In [134]:
entire_sequence_logprob = generated_logprobs.sum(dim=-1)

cache[:] = []
batch_size = entire_sequence_logprob.shape[0]
for i in range(batch_size):
    entire_sequence_logprob[i].backward(retain_graph=True)
    model.zero_grad()

cache[0].sum(axis=-1).nonzero()[0]
# woops, I need multiple copies for each input token!
# maybe I should fetch the embedding vectors from without BartModel (immediately after calling the embedding layer) and put hooks on them?


array([    0,     2,     5,     6,     7,     9,    10,    11,    13,
          15,    16,    17,    19,    20,    27,    29,    31,    32,
          36,    41,    43,    59,    71,    95,   104,   121,   135,
         155,   176,   195,   223,   231,   246,   248,   260,   309,
         315,   336,   398,   401,   467,   488,   501,   532,   586,
         598,   675,   694,   746,   752,   811,   943,  1070,  1093,
        1127,  1138,  1466,  1588,  1788,  1814,  1926,  1956,  2074,
        2096,  2266,  2447,  2546,  2619,  2885,  2946,  3072,  3073,
        3206,  3248,  3253,  3818,  3868,  4141,  4268,  4669,  4741,
        5187,  5860,  6467,  6643,  6804,  7278,  7539,  8594,  8878,
        8879,  9628, 10832, 11206, 13522, 16055, 17313, 17660, 18798,
       22645, 23066, 27193, 29531, 30680, 34124, 35590, 37529, 38027,
       39578, 39923])